In [4]:
import tensorflow as tf
import json
import numpy as np
#from tensorflow.keras import datasets, layers, models
import tensorflow as tf
import keras
import json
from keras import layers, models
import matplotlib.pyplot as plt
import datetime
from sklearn.model_selection import train_test_split

## Load in Data

In [ ]:
start = datetime.datetime.now()
with open('processed_11x11/crashes_4d_24_hour.json', 'r') as f:
    array_4d_list = json.load(f)

# Convert the nested list back to a numpy array
array_4d_24_hour = np.array(array_4d_list)

with open('processed_11x11/crashes_4d_one_hour.json', 'r') as f:
    array_4d_list = json.load(f)
array_4d_1_hour = np.array(array_4d_list) 

with open('processed_11x11/11x11_traffic_1_hour.json', 'r') as f:
    traffic_1_hour_list = json.load(f)
traffic_1_hour = np.array(traffic_1_hour_list) 

binary_output_1_hour = (array_4d_1_hour > 0).astype(int)
print("Finished Loading in:", datetime.datetime.now()-start)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x11828bd60>>
Traceback (most recent call last):
  File "/Users/ch/miniforge3/envs/finaljobsllm/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


Finished 1st item Loading in: 0:00:13.721281
Finished Loading in: 0:00:32.556148
Finished Loading in: 0:00:34.628702


## First Model - Regression

In [10]:
# Step 3: Define the model
model = models.Sequential()

# Input shape is (11, 11, 24), where 24 corresponds to the time steps
model.add(layers.Conv2D(30, (3, 3), activation='relu', input_shape=(11, 11, 24)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten the output to prepare it for Dense and LSTM layers
model.add(layers.Flatten())

# Dense layer with 24 units
model.add(layers.Dense(24, activation='relu'))

# Reshape the output to prepare it for LSTM (24 time steps, each with 1 feature)
model.add(layers.Reshape((24, 1)))  # Here, 24 timesteps, 1 feature for each timestep

# LSTM layer
model.add(layers.LSTM(64, return_sequences=False))  # LSTM layer

# Output layer for regression (this will output continuous values)
# The output shape is (batch_size, 11, 11, 1) for each image.
model.add(layers.Dense(11 * 11, activation='linear'))  # Output a vector of length 11 * 11 (121)
model.add(layers.Reshape((11, 11, 1)))  # Reshape back to (11, 11, 1) per sample

# Summarize the model
model.summary()

/Users/ch/miniforge3/envs/finaljobsllm/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 9, 9, 30)       │         6,510 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 4, 4, 30)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 2, 2, 64)       │        17,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 1, 1, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 24)             │         1,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 24, 1)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 121)            │         7,865 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_1 (Reshape)             │ (None, 11, 11, 1)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,175 (196.00 KB)

 Trainable params: 50,175 (196.00 KB)

 Non-trainable params: 0 (0.00 B)

## Second Model - Classification

In [11]:
# Step 3: Define the model
model_binary = models.Sequential()

# Input shape is (11, 11, 24), where 24 corresponds to the time steps
model_binary.add(layers.Conv2D(30, (3, 3), activation='relu', input_shape=(11, 11, 24)))
model_binary.add(layers.MaxPooling2D((2, 2)))
model_binary.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_binary.add(layers.MaxPooling2D((2, 2)))

# Flatten the output to prepare it for Dense and LSTM layers
model_binary.add(layers.Flatten())

# Dense layer with 24 units
model_binary.add(layers.Dense(24, activation='relu'))

# Reshape the output to prepare it for LSTM (24 time steps, each with 1 feature)
model_binary.add(layers.Reshape((24, 1)))  # Here, 24 timesteps, 1 feature for each timestep

# LSTM layer
model_binary.add(layers.LSTM(64, return_sequences=False))  # LSTM layer

# Output layer for binary classification
# The output shape is (batch_size, 11, 11, 1) for each image.
model_binary.add(layers.Dense(11 * 11, activation='sigmoid'))  # Sigmoid activation for binary output
model_binary.add(layers.Reshape((11, 11, 1)))  # Reshape back to (11, 11, 1) per sample

# Compile the model with binary cross-entropy loss
model_binary.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summarize the model
model_binary.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 9, 9, 30)       │         6,510 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 30)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 2, 2, 64)       │        17,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 1, 1, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 24)             │         1,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_2 (Reshape)             │ (None, 24, 1)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 121)            │         7,865 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_3 (Reshape)             │ (None, 11, 11, 1)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,175 (196.00 KB)

 Trainable params: 50,175 (196.00 KB)

 Non-trainable params: 0 (0.00 B)